In [ ]:
from __future__ import absolute_import, division, print_function
from tqdm import tqdm
from numpy.random import randn


import cv2
import pathlib
import random
import matplotlib.pyplot as plt

import tensorflow as tf
import numpy as np

from matplotlib.image import imread
from keras.preprocessing import image

In [ ]:
keras=tf.keras

In [ ]:
data_root = tf.keras.utils.get_file(
  'flower_photos','https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz',
   untar=True)

In [ ]:
data_dir = pathlib.Path(data_root)
CLASS_NAMES = np.array([item.name for item in data_dir.glob('*') if item.name != "LICENSE.txt"])
CLASS_NAMES

In [ ]:
all_images = list(data_dir.glob('*/*'))
all_images = [str(path) for path in all_images]
random.shuffle(all_images)

data_size=len(all_images)

In [ ]:
channels = 3
IMG_SIZE=150

dataset = np.ndarray(shape=(len(all_images), IMG_SIZE, IMG_SIZE,channels),
                     dtype=np.float32)

i = 0
for _file in all_images:
  image = cv2.imread(_file, 1)
  image=cv2.resize(image, (IMG_SIZE, IMG_SIZE))

  dataset[i] = image
  i += 1
print("All images to array!")

In [ ]:
dataset=dataset/255.
dataset.min(),dataset.max(),dataset.shape

In [ ]:
mean = dataset.mean(axis=(0,1,2))
std = dataset.std(axis=(0,1,2))
print(mean, std)

In [ ]:
dataset[..., 0] -= mean[0]
dataset[..., 1] -= mean[1]
dataset[..., 2] -= mean[2]

In [ ]:
dataset.min(axis=(0,1,2)),dataset.max(axis=(0,1,2))

In [ ]:
dataset[..., 0] /= std[0]
dataset[..., 1] /= std[1]
dataset[..., 2] /= std[2]

In [ ]:
dataset.min(axis=(0,1,2)),dataset.max(axis=(0,1,2))

In [ ]:
train_test_split=(int)(data_size*0.2)

x_train=all_images[train_test_split:]
x_test=all_images[:train_test_split]

y_train=all_labels[train_test_split:]
y_test=all_labels[:train_test_split]

In [ ]:

BATCH_SIZE = 32

def _parse_data(x,y):
  image = tf.read_file(x)
  image = tf.image.decode_jpeg(image, channels=3)
  image = tf.cast(image, tf.float32)
  image = (image/127.5) - 1
  image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))

  return image,y

def _input_fn(x,y):
  ds=tf.data.Dataset.from_tensor_slices((x,y))
  ds=ds.map(_parse_data)
  ds=ds.shuffle(buffer_size=data_size)


  ds = ds.repeat()

  ds = ds.batch(BATCH_SIZE)

  ds = ds.prefetch(buffer_size=AUTOTUNE)

  return ds


tf.keras.applications.vgg16.preprocess_input

In [ ]:
IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)
VGG16_MODEL=tf.keras.applications.VGG16(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')


In [ ]:
VGG16_MODEL.trainable=False
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = tf.keras.layers.Dense(len(CLASS_NAMES),activation='softmax')

In [ ]:
model = tf.keras.Sequential([
  VGG16_MODEL,
  global_average_layer,
  prediction_layer
])

In [ ]:
model.compile(optimizer=tf.train.AdamOptimizer(),
              loss=tf.keras.losses.sparse_categorical_crossentropy,
              metrics=["accuracy"])

In [ ]:
train_ds=_input_fn(x_train,y_train)
validation_ds=_input_fn(x_test,y_test)

In [ ]:
history = model.fit(train_ds,
                    epochs=100,
                    steps_per_epoch=2,
                    validation_steps=2,
                    validation_data=validation_ds)

In [ ]:
validation_steps = 20

loss0,accuracy0 = model.evaluate(validation_ds, steps = validation_steps)

print("loss: {:.2f}".format(loss0))
print("accuracy: {:.2f}".format(accuracy0))

In [ ]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
from random import randint

img_id=randint(2,len(x_test))

image = tf.read_file(x_test[img_id])
image = tf.image.decode_jpeg(image, channels=3)
image = tf.cast(image, tf.float32)
image = (image/127.5) - 1
image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))

image=tf.expand_dims(image, 0)


In [ ]:
model.predict(image)
